<a href="https://colab.research.google.com/github/baalasangar/MLAI/blob/master/13_Capstone/Userinterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pydicom

In [9]:
from tkinter import *
from tkinter import filedialog
import os
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
import pydicom as dicom
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
print(tf.__version__) 


model = tf.keras.models.load_model("./model/resnet152_model.h5")

def CLAHEprocessing(img):
  clahe = cv2.createCLAHE(clipLimit = 5)
  clahe_img = clahe.apply(img) + 30
  return clahe_img

def predict_score(dcm_image_pix):
    X_image_reshaped = [cv2.resize(img,(150,150)) for img in [dcm_image_pix]]
    normalizeimg = lambda x: (np.array(x) / 255.0).astype('float32')
    X_image_reshaped = normalizeimg(X_image_reshaped)
    X_image_reshaped = np.repeat(X_image_reshaped[..., np.newaxis], 3, -1)
    predicted = model.predict(X_image_reshaped)
    return int(round(predicted[0][0]*100,0))

def openfn():
    filename = filedialog.askopenfilename(title='open')
    print(filename)
    return filename

def open_img():
    x = openfn()
    print(x)
    ds = dicom.dcmread(x)    
    #print(ds.dtype)
    metadata = "PatientID :--{} \n \n PatientAge :--{} \n \n Gender :--{} \n \n ViewPosition :--{} \n \n BodyPartExamined :--{} \n \n".format(ds.PatientID,ds.PatientAge,ds.PatientSex,ds.ViewPosition,ds.BodyPartExamined)
    print("metadata", metadata)
    x = x.replace('.dcm', '.jpg')
    cv2.imwrite(x, ds.pixel_array)
    

    img = Image.open(x)
    img = img.resize((150, 150), Image.ANTIALIAS)
    
    img = ImageTk.PhotoImage(img)    
    panel = Label(root, image=img)
    panel.image = img
    panel.grid(row=2, column=0)
    
    metadataStr = StringVar()    
    metadataStr.set(metadata)   
    
    metadata = Label(root,textvariable=metadataStr,background="#d6d6c2")    
    metadata.grid(row=2, column=1)
    
    classificationStr = StringVar()
#     print(ds.pixel_array)
    predict_img = CLAHEprocessing(ds.pixel_array)
    
#     print(predict_img)
    prediction_score = predict_score(predict_img)
    print(prediction_score)
    affected_or_not = "NOT Affected";
    fontcolor = "green" 
    if prediction_score > 50:
        affected_or_not = "Affected"  
        fontcolor = "red"
    else:
        prediction_score = 100 - prediction_score 
    classificationStr.set("")    
    classificationStr.set("\n \n {} ( probability score {} %)".format(affected_or_not,prediction_score))   
    
    classificationLabel = Label(root,textvariable=classificationStr,background="#d6d6c2",font=("Arial", 14),fg=fontcolor)    
    classificationLabel.grid(row=3, column=1)
    
   
    
    
#     infectedLabel = StringVar()  
    
#     metadata = Label(root,textvariable=metadataStr,background="#d6d6c2")    
#     metadata.grid(row=2, column=1)
    

if __name__ == "__main__":    
    root = Tk()
    root.geometry("550x550+800+150")
    
    root.configure(background="#d6d6c2")
    root.title("Pneumonia detection") 
    root.resizable(width=True, height=True)
    
    btn = Button(root, text='Upload dcm file', command=lambda:open_img())
    btn.grid(row=1, column=0,padx=20)    
    root.mainloop()


2.6.0
D:/01_BALAJI/01_MLAI/MLAI/13_Capstone/affected.dcm
D:/01_BALAJI/01_MLAI/MLAI/13_Capstone/affected.dcm
metadata PatientID :--0a0f91dc-6015-4342-b809-d19610854a21 
 
 PatientAge :--18 
 
 Gender :--F 
 
 ViewPosition :--PA 
 
 BodyPartExamined :--CHEST 
 

52
D:/01_BALAJI/01_MLAI/MLAI/13_Capstone/normal.dcm
D:/01_BALAJI/01_MLAI/MLAI/13_Capstone/normal.dcm
metadata PatientID :--0a5c4dcb-33ac-4466-9dbf-42ed5c8ec1f0 
 
 PatientAge :--23 
 
 Gender :--M 
 
 ViewPosition :--PA 
 
 BodyPartExamined :--CHEST 
 

31
